# 梯度提升 (Gradient Boosting)

## 算法原理

梯度提升是一种序列集成方法，通过逐步拟合残差来构建强学习器：

1. **初始化**: 用一个简单模型（如均值）作为初始预测
2. **迭代拟合**: 每轮训练一个新的弱学习器来拟合当前残差
3. **累加预测**: 将所有弱学习器的预测加权求和

## 数学表达

第 $m$ 轮的模型更新：

$$F_m(x) = F_{m-1}(x) + \eta \cdot h_m(x)$$

其中：
- $F_{m-1}(x)$ 是前一轮的累积预测
- $h_m(x)$ 是第 $m$ 轮训练的弱学习器
- $\eta$ 是学习率（收缩参数）

## 关键超参数

- `n_estimators`: 弱学习器数量
- `learning_rate`: 学习率，控制每棵树的贡献
- `max_depth`: 树的最大深度，通常设为 3-5

In [ ]:
# 导入必要的库
import numpy as np
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# 设置随机种子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 1. 数据准备

使用鸢尾花数据集进行回归任务演示。

In [ ]:
# 加载数据
iris = load_iris()
X, y = iris.data, iris.target

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print(f"训练集大小: {X_train.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")
print(f"特征数量: {X.shape[1]}")

## 2. 手动实现梯度提升

通过手动迭代拟合残差来理解梯度提升的核心思想。

In [ ]:
# 第一棵树：拟合原始目标
tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=RANDOM_STATE)
tree_reg1.fit(X_train, y_train)

# 计算第一棵树的残差
residual1 = y_train - tree_reg1.predict(X_train)

# 第二棵树：拟合第一棵树的残差
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=RANDOM_STATE)
tree_reg2.fit(X_train, residual1)

# 计算第二棵树的残差
residual2 = residual1 - tree_reg2.predict(X_train)

# 第三棵树：拟合第二棵树的残差
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=RANDOM_STATE)
tree_reg3.fit(X_train, residual2)

# 组合预测：所有树的预测之和
y_pred_manual = sum(tree.predict(X_test) for tree in (tree_reg1, tree_reg2, tree_reg3))

# 评估手动实现的梯度提升
mse_manual = mean_squared_error(y_test, y_pred_manual)
r2_manual = r2_score(y_test, y_pred_manual)

print("手动实现梯度提升 (3棵树):")
print(f"  MSE: {mse_manual:.4f}")
print(f"  R²: {r2_manual:.4f}")

## 3. 使用 GradientBoostingRegressor

sklearn 的 GradientBoostingRegressor 提供了完整的梯度提升实现。

In [ ]:
# 创建梯度提升回归器
gbrt = GradientBoostingRegressor(
    n_estimators=100,          # 弱学习器数量
    learning_rate=0.1,         # 学习率
    max_depth=3,               # 树的最大深度
    min_samples_split=2,       # 内部节点再划分所需最小样本数
    min_samples_leaf=1,        # 叶子节点最少样本数
    subsample=1.0,             # 用于拟合各个基学习器的样本比例
    random_state=RANDOM_STATE
)

# 训练模型
gbrt.fit(X_train, y_train)

# 预测和评估
y_pred_gbrt = gbrt.predict(X_test)
mse_gbrt = mean_squared_error(y_test, y_pred_gbrt)
r2_gbrt = r2_score(y_test, y_pred_gbrt)

print("GradientBoostingRegressor:")
print(f"  MSE: {mse_gbrt:.4f}")
print(f"  R²: {r2_gbrt:.4f}")

## 4. 学习率的影响

学习率与树的数量之间存在权衡：
- 低学习率需要更多的树，但泛化性能通常更好
- 高学习率训练更快，但可能过拟合

In [ ]:
# 比较不同学习率
learning_rates = [0.01, 0.1, 0.5, 1.0]

print("学习率对比:")
print(f"{'学习率':>10} {'训练R²':>10} {'测试R²':>10}")
print("-" * 32)

for lr in learning_rates:
    gbrt_lr = GradientBoostingRegressor(
        n_estimators=100,
        learning_rate=lr,
        max_depth=3,
        random_state=RANDOM_STATE
    )
    gbrt_lr.fit(X_train, y_train)
    
    train_r2 = gbrt_lr.score(X_train, y_train)
    test_r2 = gbrt_lr.score(X_test, y_test)
    
    print(f"{lr:>10.2f} {train_r2:>10.4f} {test_r2:>10.4f}")

## 5. 使用 staged_predict 分析学习曲线

`staged_predict` 方法可以获取每个迭代阶段的预测，用于分析最佳树数量。

In [ ]:
# 训练一个有更多树的模型
gbrt_staged = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=2,
    random_state=RANDOM_STATE
)
gbrt_staged.fit(X_train, y_train)

# 计算每个阶段的测试误差
errors = [mean_squared_error(y_test, y_pred) 
          for y_pred in gbrt_staged.staged_predict(X_test)]

# 找到最佳树数量
best_n_estimators = np.argmin(errors) + 1
min_error = min(errors)

print(f"最佳树数量: {best_n_estimators}")
print(f"最小测试MSE: {min_error:.4f}")

# 使用最佳参数重新训练
gbrt_best = GradientBoostingRegressor(
    n_estimators=best_n_estimators,
    learning_rate=0.1,
    max_depth=2,
    random_state=RANDOM_STATE
)
gbrt_best.fit(X_train, y_train)
print(f"最佳模型 R²: {gbrt_best.score(X_test, y_test):.4f}")

## 6. 单元测试验证

In [ ]:
def test_gradient_boosting():
    """梯度提升功能测试"""
    
    # 测试1: 模型应该正确训练
    assert hasattr(gbrt, 'estimators_'), "模型未正确训练"
    
    # 测试2: 树的数量应该正确
    assert len(gbrt.estimators_) == 100, "树的数量不正确"
    
    # 测试3: R² 分数应该在合理范围
    assert r2_gbrt >= 0.9, f"R² 分数过低: {r2_gbrt}"
    
    # 测试4: 手动实现应该有合理性能
    assert r2_manual >= 0.8, f"手动实现 R² 过低: {r2_manual}"
    
    # 测试5: 预测结果形状应该正确
    assert y_pred_gbrt.shape == y_test.shape, "预测结果形状不正确"
    
    # 测试6: 最佳树数量应该在合理范围
    assert 1 <= best_n_estimators <= 200, "最佳树数量不在预期范围"
    
    print("所有测试通过!")

test_gradient_boosting()

## 总结

### 梯度提升的优势

1. **高预测精度**: 在结构化数据上通常表现最佳
2. **内置正则化**: 通过学习率和树深度控制过拟合
3. **特征重要性**: 自动计算特征重要性

### 梯度提升的局限

1. **训练较慢**: 必须串行训练，无法并行
2. **参数敏感**: 需要仔细调参
3. **对异常值敏感**: 特别是使用平方损失时

### 调参建议

1. 先固定 `learning_rate=0.1`，调整 `n_estimators`
2. 使用 `staged_predict` 或早停找到最佳迭代次数
3. 适当降低 `learning_rate`（如 0.01-0.1）并增加 `n_estimators`
4. `max_depth` 通常设为 3-5